This notebook shows how to train a neural network using images and memory. Note that the dirs variable shoould be the path to the data folder containing a folder called screenshots and another folder called states. There should be respectively images and memory files in each folder.

# Preparation

In [ ]:
%pylab inline
from keras import layers
from keras.models import Model
from sklearn.metrics import average_precision_score
from glob import glob

In [ ]:
dirs = '' #Path to the data folder

#The corresponding image and memory file should have the same numeric name
#indicating the frame count. Run "Working with Human Data" notebook for example
img_paths = sorted(glob(dirs + '/screenshots/*'),
                   key=lambda name: int(name[len(dirs)+13:-4]))
mem_paths = sorted(glob(dirs + '/states/*'),
                   key=lambda name: int(name[len(dirs)+8:-4]))

img_num = len(img_paths)
if img_num != len(mem_paths):
    raise Exception('The amount of screenshots and states are different')
    
img_height, img_width, img_channel = imread(img_paths[0]).shape
ram_size = 4096

print('Number of images:', img_num)
print('Image shape: (%i, %i, %i)' % (img_height, img_width, img_channel))

# Create Keras Model

In [ ]:
#Construct neural network

def make_layers(input_layer, filters, window_size, strides, activation, dropout_rate):
    x = layers.Conv2D(filters, window_size, strides=strides, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Dropout(dropout_rate)(x)
    return x

input_img = layers.Input(shape=(img_height, img_width, img_channel))

x = make_layers(input_img, 16, 3, 2, 'elu', 0.5)
x = make_layers(x, 32, 3, 2, 'elu', 0.5)
x = make_layers(x, 64, 3, 2, 'elu', 0.5)
x = make_layers(x, 64, 3, 2, 'elu', 0.5)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu', name='Embedded')(x)
embedded = x
x = layers.Dropout(0.5)(x)

x = layers.Dense(ram_size * 4)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Reshape((ram_size, 4))(x)
x = layers.Dense(64)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='softmax')(x)

model = Model(input_img, x)
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

# Training

In [ ]:
#An array of values to an array of one hot vectors
def byte_values_to_categorical(y):
    y = y.ravel()
    n = y.shape[0]
    categorical = np.zeros((n, 256), dtype=np.uint8)
    categorical[arange(n), y] = 1
    return categorical

#An array of one hot vectors to an array of values
def categorical_to_byte_values(y):
    return argmax(y, axis=2).astype(np.uint8)

#Training data generator
def gen(arr, batch_size):
    i = 0
    total_num = len(arr)
    while True:
        temp_arr = arr[i * batch_size:(i + 1) * batch_size]
        length = len(temp_arr)
        temp_screenshots = np.zeros((length, img_height, img_width, img_channel))
        temp_states = np.zeros((length, ram_size))
        for j in range(length):
            temp_num = temp_arr[j]
            temp_screenshots[j] = imread(img_paths[temp_num])
            with open(mem_paths[temp_num], 'rb') as f:
                temp_states[j] = np.frombuffer(f.read()[:ram_size], dtype=np.uint8)
        temp_real_states = byte_values_to_categorical(temp_states.astype(np.uint8)).reshape(-1, ram_size, 256)
        i += 1
        if i * batch_size >= total_num:
            i = 0
        yield temp_screenshots, temp_real_states
        
batch_size = 16
this_arr = arange(1, img_num)
np.random.shuffle(this_arr)
split = img_num // 10
validation_arr = this_arr[8 * split:9 * split]
test_arr = this_arr[9 * split:]
train_arr = this_arr[:8 * split]
history = model.fit_generator(gen(train_arr, batch_size),
                              steps_per_epoch=int(len(this_arr) / batch_size) + 1,
                              epochs=20,
                              validation_data=gen(validation_arr, batch_size),
                              validation_steps=int(len(validation_arr) / batch_size) + 1,
                              )

# Show mean average precision

In [ ]:
def mAP(vectors, t):
    special_frames = list(range(0, len(vectors), 60))
    results = []
    normalized_vectors = vectors / norm(vectors, axis=1).reshape((-1, 1))
    
    for special_frame in special_frames:
        y_true = array([abs(i - special_frame) <= t for i in range(len(vectors))])
        target = normalized_vectors[special_frame]
        scores = dot(normalized_vectors, target)
        y_true[special_frame] = False
        scores[special_frame] = min(scores)
        results.append(average_precision_score(y_true, scores))
    return mean(results)

#Get em
embedded_model = Model(model.input, embedded)
this_arr = arange(img_num)
embedding = embedded_model.predict_generator(gen(this_arr, batch_size),
                                             steps=int(len(this_arr) / batch_size) + 1,
                                             verbose=1)

#Geeting mean average precision
print('2 Seconds:', mAP(embedding, 1))
print('10 Seconds:', mAP(embedding, 5))
print('60 Seconds:', mAP(embedding, 30))

#Save the full model and embedding model
model.save('full_model.h5')
embedded_model.save('embedding_model.h5')